In [5]:
import math
import numpy as np
from tkinter import Tk, Canvas
import random
import time

# Scale factor to enlarge the map
scale = 10  

# Constants
MAP_WIDTH = 140 * scale  # Width of the map
MAP_HEIGHT = 43 * scale  # Height of the map
PARTICLE_COUNT = 500
PARTICLE_RADIUS = 4  # Radius of the particles
y_offset = 5  # Push map down
offset = 5
speed = 15
dt = 0.01
door_capacity = 5
# Wall coordinates
x1 = [0, 0, 0, 12]
y1 = [9, 28, 28, 28]

x2 = [17, 30, 30, 30, 30, 104]
y2 = [28, 28, 28, 40, 40, 40]

x3 = [109, 140, 140, 140]
y3 = [40, 40, 40, 10]

x4 = [140, 140, 140, 104, 104, 104, 104, 67, 67, 67, 67, 58, 58, 58, 58, 44, 44, 44, 44, 0]
y4 = [5, 0, 0, 0, 0, 30, 30, 30, 30, 6, 6, 6, 6, 20, 20, 20, 20, 9, 9, 9]

door_1_y = 28
door_1_x_min = 12
door_1_x_max = 17

door_2_y = 40
door_2_x_min = 104
door_2_x_max = 109

door_3_x = 140
door_3_y_min = 5
door_3_y_max = 10

targets_x=[106,35,32]
targets_y=[35,35,27]

def get_nearest_door(px, py):
    door1_x_center = (door_1_x_min + door_1_x_max) / 2
    door1_dis = math.sqrt((px - door1_x_center) ** 2 + (py - door_1_y) ** 2)
    
    door2_x_center = (door_2_x_min + door_2_x_max) / 2
    door2_dis = math.sqrt((px - door2_x_center) ** 2 + (py - door_2_y) ** 2)
    
    door3_y_center = (door_3_y_min + door_3_y_max) / 2
    door3_dis = math.sqrt((px - door_3_x) ** 2 + (py - door3_y_center) ** 2)
    
    distances = [door1_dis, door2_dis, door3_dis]
    return distances.index(min(distances)) + 1  # Return door number (1, 2, or 3)

def get_target(px,py,t_door):
    if t_door==1:
        if 0 <= px <= 67 and py <=28:
            return 14.5,28
        elif 30 <= px <= 67 and py >=28:
            return 32,25
        elif 67 < px <= 104 and py >=28:
            return 67,35
        else:
            return 104,35
    elif t_door== 2:
        if 0 <= px <= 30 or 45 <= px <= 67 and py <=28:
            return 67,35
        elif 31 <= px <= 44 and py <=28:
            return 44,35
        else:
            return 106.5,40
    else:
        if 0 <= px <= 30:
            return 31,28
        elif 30 < px <= 44:
            return 44,35
        elif 44 < px <= 67:
            return 67,35
        elif 67 < px <= 104:
            return 105,35
        else:
           return 140,7.5

def check_overlap(x, y, index, x_values, y_values):

    for i in range(len(x_values)):
        if i != index and evacuated_particles[i] == False: 
            distance = np.sqrt((x - x_values[i])**2 + (y - y_values[i])**2)
            if distance <= 0.5:  # 2 * PARTICLE_RADIUS is the combined radius of two particles
                return True
    return False

def move_toward_target_1(px, py, door):
    door_x,door_y = get_target(px,py,door)
    dy = door_y - py
    dx = door_x - px

    dist = np.sqrt(dx**2 + dy**2)
    if dist != 0:
        return (speed * dx / dist, speed * dy / dist)
    else:
        return (0, 0)  # No movement if at the door



# Functions for scaling and flipping coordinates
def scale_and_offset(coords, scale, offset=5):
    return [(coord * scale) + offset for coord in coords]

def flip_y_coords(coords, map_height, offset=10):
    return [map_height - coord for coord in coords]

# Scale and offset wall coordinates
x1_scaled = scale_and_offset(x1, scale)
y1_scaled = flip_y_coords(scale_and_offset(y1, scale, y_offset), MAP_HEIGHT)

x2_scaled = scale_and_offset(x2, scale)
y2_scaled = flip_y_coords(scale_and_offset(y2, scale, y_offset), MAP_HEIGHT)

x3_scaled = scale_and_offset(x3, scale)
y3_scaled = flip_y_coords(scale_and_offset(y3, scale, y_offset), MAP_HEIGHT)

x4_scaled = scale_and_offset(x4, scale)
y4_scaled = flip_y_coords(scale_and_offset(y4, scale, y_offset), MAP_HEIGHT)

# Initialize Tkinter root and canvas
root = Tk()
canvas = Canvas(root, width=MAP_WIDTH + 10, height=MAP_HEIGHT + 10, bg="white")
canvas.pack()

# Function to draw walls
def draw_walls(x_coords, y_coords):
    for i in range(len(x_coords) - 1):
        canvas.create_line(
            x_coords[i], y_coords[i], x_coords[i + 1], y_coords[i + 1],
            fill="black", width=2
        )

# Draw the walls
draw_walls(x1_scaled, y1_scaled)
draw_walls(x2_scaled, y2_scaled)
draw_walls(x3_scaled, y3_scaled)
draw_walls(x4_scaled, y4_scaled)

# Generate random particles within defined areas
x_values = np.random.uniform(1, 139, PARTICLE_COUNT)
y_values = np.zeros(PARTICLE_COUNT)

evacuated_particles = np.zeros(PARTICLE_COUNT, dtype=bool)
door_usage = {"door_1": 0, "door_2": 0, "door_3": 0}
door_usage_T = {"door_1": 0, "door_2": 0, "door_3": 0}

target_door = np.random.choice([1,2,3], size=PARTICLE_COUNT)

for i in range(PARTICLE_COUNT):
    x = x_values[i]
    if 0 <= x <= 44:
        y_values[i] = np.random.uniform(10, 27)
    elif 30 <= x <= 67:
        y_values[i] = np.random.uniform(28, 39)
    elif 67 <= x <= 104:
        y_values[i] = np.random.uniform(31, 39)
    elif 104 <= x <= 140:
        y_values[i] = np.random.uniform(1, 39)
    elif 44 <= x <= 58:
        y_values[i] = np.random.uniform(20, 28)
    elif 58 <= x <= 67:
        y_values[i] = np.random.uniform(6, 28)

nearest_target = [get_nearest_door(px, py) for px, py in zip(x_values, y_values)]


def scale_it(x_val, y_val):
    x_scaled = scale_and_offset(x_val, scale)
    y_scaled = flip_y_coords(scale_and_offset(y_val, scale, y_offset), MAP_HEIGHT)
    return x_scaled, y_scaled

def move_toward_target(px, py, door):
    if door == 1:
        dy = door_1_y - py
        dx = (door_1_x_min + door_1_x_max) / 2 - px
    elif door == 2:
        dy = door_2_y - py
        dx = (door_2_x_min + door_2_x_max) / 2 - px
    elif door == 3:
        dx = door_3_x - px
        dy = (door_3_y_min + door_3_y_max) / 2 - py
    dist = np.sqrt(dx**2 + dy**2)
    if dist != 0:
        return (speed * dx / dist, speed * dy / dist)
    else:
        return (0, 0)  # No movement if at the door


# Update particle positions function
def update_particle_positions(target_door):
    global x_values, y_values, evacuated_particles, door_usage
    door_usage["door_1"] = 0
    door_usage["door_2"] = 0
    door_usage["door_3"] = 0
    for i in range(PARTICLE_COUNT):
        if not evacuated_particles[i]:
            # Calculate velocity towards the target
        # Calculate velocity towards the target
            # if target_door[i] == 1 or target_door[i] == 2:
            #vx, vy= move_toward_target_1(x_values[i], y_values[i], target_door[i])
            vx, vy= move_toward_target_1(x_values[i], y_values[i], target_door[i])
            # else:
            #     vx, vy = move_toward_target(x_values[i], y_values[i], target_door[i])

            # Update position
            px= x_values[i] +vx * dt
            py=y_values[i]+vy * dt
            # if check_overlap(px, py, i, x_values, y_values):
            #     x_values[i] += vx * dt *np.random.rand()
            #     y_values[i] += vy * dt*np.random.rand()
            # else:
            x_values[i] += vx * dt
            y_values[i] += vy * dt
            # Check for evacuation through doors
            if (target_door[i] == 1 and door_usage["door_1"] < door_capacity 
                    and door_1_x_min <= x_values[i] <= door_1_x_max and y_values[i] >= door_1_y):
                evacuated_particles[i] = True
                door_usage["door_1"] += 1
                door_usage_T["door_1"] += 1
            elif (target_door[i] == 2 and door_usage["door_2"] < door_capacity 
                  and door_2_x_min <= x_values[i] <= door_2_x_max and y_values[i] >= door_2_y):
                evacuated_particles[i] = True
                door_usage["door_2"] += 1
                door_usage_T["door_2"] += 1
            elif (target_door[i] == 3 and door_usage["door_3"] < door_capacity 
                  and x_values[i] >= door_3_x and door_3_y_min <= y_values[i] <= door_3_y_max):
                evacuated_particles[i] = True
                door_usage["door_3"] += 1
                door_usage_T["door_3"] += 1

# Particle initialization and drawing on canvas
initial_x, initial_y = scale_it(x_values, y_values)
particles = []
for i in range(PARTICLE_COUNT):
    if target_door[i] == 1:
        color = "blue"
    elif target_door[i] == 2:
        color = "green"
    else:
        color = "red"
    particles.append(
        canvas.create_oval(
            initial_x[i] - PARTICLE_RADIUS, initial_y[i] - PARTICLE_RADIUS,
            initial_x[i] + PARTICLE_RADIUS, initial_y[i] + PARTICLE_RADIUS,
            fill=color
        )
    )

# Simulation loop
step = 0
while np.sum(evacuated_particles) < PARTICLE_COUNT:
    #update_particle_positions(nearest_target)
    update_particle_positions(target_door)
    print(np.sum(evacuated_particles))
    x, y = scale_it(x_values, y_values)

    # Update canvas
    for j, particle in enumerate(particles):
        if not evacuated_particles[j]:
            canvas.coords(
                particle,
                x[j] - PARTICLE_RADIUS, y[j] - PARTICLE_RADIUS,  # Flip the Y-coordinate for particles
                x[j] + PARTICLE_RADIUS, y[j] + PARTICLE_RADIUS,
            )
        else:
            canvas.coords(particle, 0, 0, 0, 0)  # Hide evacuated particles

    root.update()
    #time.sleep(0.001)  # A more reasonable delay for updates

# Run the Tkinter main loop
root.mainloop()


0
0
0
0
0
0
0
0
0
0
0
1
1
1
1
1
2
2
2
2
2
3
3
3
5
5
6
6
6
6
6
7
7
7
8
8
8
9
9
11
11
11
12
13
13
13
13
13
14
15
17
17
18
19
19
20
20
21
21
21
23
26
26
26
26
27
28
28
29
29
30
32
33
33
34
36
37
37
38
38
39
40
42
43
43
44
45
45
46
47
47
48
50
51
51
52
52
53
54
55
55
57
57
59
60
61
61
61
61
62
64
64
64
65
67
70
70
73
73
73
73
73
73
74
76
77
77
79
79
79
79
80
81
83
83
84
84
84
84
86
86
86
86
86
87
88
89
89
90
90
90


TclError: invalid command name ".!canvas"